In [1]:
import re
import bs4
import json
import pandas as pd
from tqdm import tqdm
from glob import glob
from bs4 import BeautifulSoup

from blog_fake_review.parser import extract_article_body, refine_paragraph_tag, extract_paragraph, extract_additional_info

In [2]:
origins = glob("./data/origin/*.json")
origins

['./data/origin/홍대 회식 맛집.json',
 './data/origin/홍대 데이트 맛집.json',
 './data/origin/성수 데이트 맛집.json',
 './data/origin/압구정 회식 맛집.json',
 './data/origin/강남역 데이트 맛집.json',
 './data/origin/영등포 회식 맛집.json',
 './data/origin/성수 회식 맛집.json',
 './data/origin/압구정 데이트 맛집.json',
 './data/origin/강남역 회식 맛집.json',
 './data/origin/범계 회식 맛집.json',
 './data/origin/범계 데이트 맛집.json',
 './data/origin/영등포 데이트 맛집.json']

In [19]:
parsed_list = glob("./data/semi-structured/*.json")
# parsed_list = [p.split('/')[-1] for p in parsed_list]
parsed_list

['./data/semi-structured/홍대 회식 맛집.json',
 './data/semi-structured/홍대 데이트 맛집.json',
 './data/semi-structured/성수 데이트 맛집.json',
 './data/semi-structured/압구정 회식 맛집.json',
 './data/semi-structured/강남역 데이트 맛집.json',
 './data/semi-structured/영등포 회식 맛집.json',
 './data/semi-structured/성수 회식 맛집.json',
 './data/semi-structured/압구정 데이트 맛집.json',
 './data/semi-structured/강남역 회식 맛집.json',
 './data/semi-structured/범계 회식 맛집.json',
 './data/semi-structured/범계 데이트 맛집.json',
 './data/semi-structured/영등포 데이트 맛집.json']

In [20]:
for path in parsed_list:
    file_name = path.split('/')[-1]
    
#     if file_name in parsed_list: continue
        
    with open(path, "r", encoding='utf-8') as f:
        blog = json.load(f)
        
    print(len(blog))

888
855
985
914
858
932
981
945
950
945
902
927


In [4]:
comment_code = re.compile(r'<!--.*?-->')

In [5]:
for path in origins:
    file_name = path.split('/')[-1]
    
#     if file_name in parsed_list: continue
        
    with open(path, "r", encoding='utf-8') as f:
        blog = json.load(f)

    result = []
    for row in tqdm(blog, desc=file_name.split('.')[0]):
        data = {
            "blog_name": row["blog_name"],
            "date": row["date"],
            "url": row["url"],
            "title": row["title"]
        }
        
        soup = BeautifulSoup(comment_code.sub('', row['origin_source']), 'html.parser')
        version, body_tag = extract_article_body(soup)
        
        if not body_tag: continue
            
        tags = extract_paragraph(body_tag, version)
        
        data["contents"] = sum([refine_paragraph_tag(tag) for tag in tags], [])
        
        data["additional"] = extract_additional_info(soup)
        result.append(data)
    
    with open(f'./data/semi-structured/{file_name}', 'w', encoding='utf-8') as f:
        json.dump(result, f)

영등포 데이트 맛집: 100%|██████████| 979/979 [01:16<00:00, 12.74it/s]


In [6]:
parsed_list = glob("./data/semi-structured/*.json")
parsed_list

['./data/semi-structured/홍대 회식 맛집.json',
 './data/semi-structured/홍대 데이트 맛집.json',
 './data/semi-structured/성수 데이트 맛집.json',
 './data/semi-structured/압구정 회식 맛집.json',
 './data/semi-structured/강남역 데이트 맛집.json',
 './data/semi-structured/영등포 회식 맛집.json',
 './data/semi-structured/성수 회식 맛집.json',
 './data/semi-structured/압구정 데이트 맛집.json',
 './data/semi-structured/강남역 회식 맛집.json',
 './data/semi-structured/범계 회식 맛집.json',
 './data/semi-structured/범계 데이트 맛집.json',
 './data/semi-structured/영등포 데이트 맛집.json']

In [30]:
for path in parsed_list:
    with open(path, "r", encoding='utf-8') as f:
        blog = json.load(f)
    for article in blog:
        contents = []
        contents_meta = {}
        for i, para in enumerate(article['contents']):
            para = {
                'no': i + 1,
                'info': para['info'],
                'content': para['content']
            }
            
            contents_meta[para['info']] = contents_meta.setdefault(para['info'], 0) + 1
            contents.append(para)
        
        article['contents_meta'] = contents_meta
        article['contents'] = contents
        article['id'] = '.'.join(article['url'].split('/')[-2:])
        

    with open(path, 'w', encoding='utf-8') as f:
        json.dump(blog, f)

In [31]:
with open(parsed_list[0], "r", encoding='utf-8') as f:
    blog = json.load(f)

In [32]:
blog[0]

{'blog_name': 'shuen',
 'date': '20230225',
 'url': 'https://blog.naver.com/shuenmama/223027650182',
 'title': '<b>홍대</b> 고기집 데이트 쟁반한상 삼겹살 <b>맛집</b> <b>회식</b> 쟁반집8292',
 'contents': [{'no': 1,
   'info': 'text',
   'content': '홍대에서 데이트 하기로 한 주말\n진짜 간만에 홍대\n전에 항상 세우던 공영주차장이 막혀있어서\n근처 유료주차장에 주차했다\n\n엄청나게 막혀도 홍대 데이트는 포기 못하지 :)\n사람이 꽉찬 북적한 홍대거리 너무 좋다\n새로생긴 가게들도 많고\n구경할것도 많고 얼마나 많이 돌아다녔는지\n나오기 전 늦은 아침을 간단하게 먹었는대도\n고기가 먹고싶어진다 :)\n\n구경하면서 봐뒀던\n#홍대고기집 쟁반집8292\n12시 오픈이라 주변에서 몇바퀴 더 돌고있었다'},
  {'no': 2,
   'info': 'img',
   'content': 'https://postfiles.pstatic.net/MjAyMzAyMjVfOCAg/MDAxNjc3MzE3ODY2MjYz.xVfavTXtn1K6evITcB1lbTdljYyr34S0bTAG7K3H0W8g.5SpztDyBDQJ_PVupIW_UAXfKR7luiVE6Nmhc-e3orA4g.JPEG.shuenmama/IMG_2027.jpg?type=w773'},
  {'no': 3,
   'info': 'text',
   'content': '오픈전 지나갈때는 사람이 없었는데\n오픈시간 12시가 다가오면 #홍대고기집맛집 앞에\n기다리는 사람들이 하나둘 생겼다\n\n홍대를 이른 시간에 둘러보다 보면\n어떤집은 오픈하기 전부터 가게앞에 사람들이\n대기를 하고있고\n어떤집은 오픈 후에도 널널한 모습이기도 해서\n#홍대 고기집 맛집 고르는 기준이 되기도 한다'},
  {'no': 4,
   'info': 'img',
   'content': '